<a href="https://colab.research.google.com/github/Sandeepkumaramgothu/ASE_practice/blob/main/Spect_2_Gun_Other.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install --upgrade keras
!pip install --upgrade keras tensorflow
import os
import math
import numpy as np
import librosa
import json
import pandas as pd
from termcolor import colored
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
!pip install tensorboard
!pip install visualkeras
import visualkeras
from keras.utils import plot_model
from datetime import datetime
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn import svm
from sklearn.svm import SVC
import pickle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from IPython.display import Audio
from scipy.io import wavfile
import scipy
import soundfile as sf
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report
import random



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.4/997.4 kB 10.8 MB/s eta 0:00:00


In [4]:
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from sklearn.metrics import classification_report, confusion_matrix


In [5]:

def create_mel_spectrogram(file_path, save_path, n_mels=128, n_fft=2048, hop_length=512, sr=16000):
    """ This function generates a mel spectrogram from a sound file and saves it as an image. """
    y, sr = librosa.load(file_path, sr=sr)

    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, n_fft=n_fft, hop_length=hop_length)
    S_DB = librosa.amplitude_to_db(S, ref=np.max)

    plt.figure(figsize=(10, 4))
    librosa.display.specshow(S_DB, sr=sr, hop_length=hop_length, x_axis='time', y_axis='mel')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Mel spectrogram')
    plt.tight_layout()

    plt.savefig(save_path)
    #plt.savefig(save_path, bbox_inches='tight', pad_inches=0)
    print(f'Mel spectrogram saved as {save_path}\n')
    plt.close()

def prepare_dataset(sound_dir, image_dir):
    """ Processes each category in train and test directories to create spectrogram images. """
    for set_type in ['Train', 'Test']:
        set_sound_dir = os.path.join(sound_dir, set_type)
        set_image_dir = os.path.join(image_dir, set_type)
        categories = os.listdir(set_sound_dir)
        for category in categories:
            filecounter=0
            print(f'Processing {category} category... in {set_type} ')
            cat_sound_dir = os.path.join(set_sound_dir, category)
            cat_image_dir = os.path.join(set_image_dir, category)
            os.makedirs(cat_image_dir, exist_ok=True)
            sound_files = [f for f in os.listdir(cat_sound_dir) if f.endswith('.wav') or f.endswith('.mp3')]
            for sound_file in sound_files:
                file_path = os.path.join(cat_sound_dir, sound_file)
                print(f'Processing {file_path}')
                image_path = os.path.join(cat_image_dir, sound_file.replace('.wav', '.png'))
                create_mel_spectrogram(file_path, image_path)
                filecounter+=1
            print(f"Processed {category} with {filecounter} files \n\n")




In [ ]:
base_dir = '/content/drive/MyDrive/DataSet/Dataset_Version_2'
sound_dir = os.path.join(base_dir, 'Sound')
image_dir = os.path.join(base_dir, 'Images')

prepare_dataset(sound_dir, image_dir)

In [ ]:
num_classes = 3

n_fft = 2048
hop_length = 512
sr = 16000
T = 1

hop_size_sec = hop_length / sr
audio_length_samples = T * sr

height = 128
width = 1 + int((audio_length_samples - n_fft) / hop_length)

channels = 3

batch_size = 32

print(f"height: {height}")
print(f"width: {width}")
print(f"channels: {channels}")


train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator()



train_generator = train_datagen.flow_from_directory(
        os.path.join(image_dir, 'Train'),
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        os.path.join(image_dir, 'Test'),
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical')

model = tf.keras.Sequential()

# Convolutional layers
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(height, width, channels)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))

# Flatten layer to feed into fully connected layers
model.add(tf.keras.layers.Flatten())

# Dense layers
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])

epochs = 150
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    train_generator.reset()

    model.fit(train_generator,steps_per_epoch=len(train_generator),epochs=1,verbose=1)

loss, accuracy = model.evaluate(test_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

model.save('el-amadi_cnn_standardized_unknown.keras')

height: 128
width: 28
channels: 3
Found 270 images belonging to 3 classes.
Found 30 images belonging to 3 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/150


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 50s 5s/step - accuracy: 0.3070 - loss: 27.8725
Epoch 2/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 553ms/step - accuracy: 0.3876 - loss: 3.2899
Epoch 3/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 467ms/step - accuracy: 0.3630 - loss: 1.2699
Epoch 4/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 457ms/step - accuracy: 0.5231 - loss: 0.9954
Epoch 5/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 526ms/step - accuracy: 0.5602 - loss: 0.9431
Epoch 6/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 448ms/step - accuracy: 0.5814 - loss: 0.9164
Epoch 7/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 620ms/step - accuracy: 0.5954 - loss: 0.9649
Epoch 8/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 457ms/step - accuracy: 0.5997 - loss: 0.8769
Epoch 9/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 497ms/step - accuracy: 0.6822 - loss: 0.8059
Epoch 10/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 479ms/step - accuracy: 0.6572 - loss: 0.8101
Epoch 11/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 449ms/step - accuracy: 0.7017 - loss: 0.7632
Epoch 12/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 617ms/step - accuracy: 0.6655 - los

In [11]:
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator

num_classes = 3
height = 128
n_fft = 2048
hop_length = 512
sr = 16000
T = 1

hop_size_sec = hop_length / sr
audio_length_samples = T * sr

width = 1 + int((audio_length_samples - n_fft) / hop_length)

channels = 3

batch_size = 32

model = tf.keras.models.load_model('/content/el-amadi_cnn_standardized_unknown.keras')


test_datagen = ImageDataGenerator()

test_generator = test_datagen.flow_from_directory(
        os.path.join(image_dir, 'Test'),
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

loss, accuracy = model.evaluate(test_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Get true class labels
true_classes = test_generator.classes

# Compute confusion matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("\nConfusion Matrix:")
print(conf_matrix)

# Get class labels
class_labels = list(test_generator.class_indices.keys())

# Calculate accuracy for each class
class_accuracy = {}
for i, class_label in enumerate(class_labels):
    correct_indices = np.where(true_classes == i)[0]
    correct_predictions = np.sum(predicted_classes[correct_indices] == i)
    total_samples = len(correct_indices)
    class_accuracy[class_label] = correct_predictions / total_samples if total_samples > 0 else 0.0

# Print accuracy for each class
print("\nAccuracy for each class:")
for class_label in class_labels:
    print(f"{class_label}: {class_accuracy[class_label]:.4f}")

# Print classification report (includes precision, recall, F1-score)
print("\nClassification Report:")
print(classification_report(true_classes, predicted_classes, target_names=class_labels))

Found 30 images belonging to 3 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 729ms/step - accuracy: 1.0000 - loss: 4.1327e-04
Test Loss: 0.00041326656355522573
Test Accuracy: 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 431ms/step

Confusion Matrix:
[[10  0  0]
 [ 0 10  0]
 [ 0  0 10]]

Accuracy for each class:
AK-12: 1.0000
AK-47: 1.0000
M-4: 1.0000

Classification Report:
              precision    recall  f1-score   support

       AK-12       1.00      1.00      1.00        10
       AK-47       1.00      1.00      1.00        10
         M-4       1.00      1.00      1.00        10

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



In [ ]:
#This VERSION is of GPT

#This VERSION is of GPT

In [17]:
#train_datagen = ImageDataGenerator(rescale=1./255)
#test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    os.path.join(image_dir, 'Train'),
    target_size=(128, 28),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    os.path.join(image_dir, 'Test'),
    target_size=(128, 28),
    batch_size=32,
    class_mode='categorical',
    shuffle=False

)



Found 270 images belonging to 3 classes.
Found 30 images belonging to 3 classes.


In [18]:


model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 28, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy','precision','recall','f1_score'])

model.fit(train_generator, epochs=50, validation_data=test_generator)




/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50


9/9 ━━━━━━━━━━━━━━━━━━━━ 8s 597ms/step - accuracy: 0.3614 - f1_score: 0.3428 - loss: 35.5230 - precision: 0.3607 - recall: 0.3603 - val_accuracy: 0.2667 - val_f1_score: 0.1991 - val_loss: 4.6083 - val_precision: 0.2667 - val_recall: 0.2667
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 690ms/step - accuracy: 0.4225 - f1_score: 0.3537 - loss: 3.9547 - precision: 0.4290 - recall: 0.4129 - val_accuracy: 0.3667 - val_f1_score: 0.3027 - val_loss: 1.2011 - val_precision: 0.4545 - val_recall: 0.3333
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 489ms/step - accuracy: 0.3996 - f1_score: 0.3914 - loss: 1.3144 - precision: 0.4384 - recall: 0.3109 - val_accuracy: 0.3667 - val_f1_score: 0.2906 - val_loss: 1.1694 - val_precision: 0.5000 - val_recall: 0.2333
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 582ms/step - accuracy: 0.4691 - f1_score: 0.4468 - loss: 1.0738 - precision: 0.4757 - recall: 0.2255 - val_accuracy: 0.3667 - val_f1_score: 0.3468 - val_loss: 1.0399 - val_precision: 0.5000 - val_recall: 0.0333
Epoch 5/50

In [19]:
results = model.evaluate(test_generator)
print(f"Test Loss: {results[0]}")
print(f"Test Accuracy: {results[1]}")

print(results)

predictions = model.predict(test_generator)

# Get predicted classes directly from predictions
y_pred = np.argmax(predictions, axis=1)
y_true = test_generator.classes

print("predicted values : ",y_pred)
print("True values      : ",y_true)


print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))

print("\nClassification Report:")
print(classification_report(y_true, y_pred))



predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Get true class labels
true_classes = test_generator.classes

# Compute confusion matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("\nConfusion Matrix:")
print(conf_matrix)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 397ms/step - accuracy: 0.9667 - f1_score: 0.9666 - loss: 0.1785 - precision: 0.9655 - recall: 0.9333
Test Loss: 0.17852824926376343
Test Accuracy: 0.9666666388511658
[0.17852824926376343, 0.9666666388511658, 0.9655172228813171, 0.9333333373069763, <tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.9473683, 0.9523809, 1.       ], dtype=float32)>]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 463ms/step
predicted values :  [0 0 1 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2]
True values      :  [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2]

Confusion Matrix:
[[ 9  1  0]
 [ 0 10  0]
 [ 0  0 10]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.90      0.95        10
           1       0.91      1.00      0.95        10
           2       1.00      1.00      1.00        10

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg